In [11]:
# ! pip install shap
import shap
import pandas as pd
import numpy as np

estr = pd.read_csv('stall2.csv')
vstr = pd.read_csv('visiter2.csv')

In [12]:
vstr.select_dtypes('object').columns

Index(['visitor_profession', 'visitor_role', 'visitor_preferrance'], dtype='object')

In [13]:
vstr.head()

,visitor_profession,visitor_role,visitor_preferrance
0,Engineer,Businessman,Cutting Machine
1,Student/Teacher,Businessman,Cutting Machine
2,Engineer,Businessman,Sugar Machine
3,Management,Job,Sugar Machine
4,Student/Teacher,Job,Sugar Machine


In [14]:
vstr[vstr.select_dtypes('object').columns].nunique().reset_index(name='cardinality')

,index,cardinality
0,visitor_profession,3
1,visitor_role,2
2,visitor_preferrance,4


In [15]:
vstr.head()

,visitor_profession,visitor_role,visitor_preferrance
0,Engineer,Businessman,Cutting Machine
1,Student/Teacher,Businessman,Cutting Machine
2,Engineer,Businessman,Sugar Machine
3,Management,Job,Sugar Machine
4,Student/Teacher,Job,Sugar Machine


In [20]:
import tensorflow as tf
def generate_categorical_feature_tf(categorical_features,num_features,data):
    models= []
    inputs = []
    for cat in categorical_features:
        vocab_size = data[cat].nunique()
        inpt = tf.keras.layers.Input(shape=(1,),name='input_'+'_'.join(cat.split(' ')))
        inputs.append(inpt)
        embed = tf.keras.layers.Embedding(vocab_size,200,\
                                          trainable=True,embeddings_initializer=tf.initializers.random_normal)(inpt)
        embed_rehsaped =tf.keras.layers.Reshape(target_shape=(200,))(embed)
        models.append(embed_rehsaped)
    num_input = tf.keras.layers.Input(shape=(len(num_features)),\
                                      name='input_number_features')
    inputs.append(num_input)
    models.append(num_input)
    merge_models= tf.keras.layers.concatenate(models)
    pre_preds = tf.keras.layers.Dense(1000)(merge_models)
    pre_preds = tf.keras.layers.BatchNormalization()(pre_preds)
    pre_preds = tf.keras.layers.Dense(1000)(pre_preds)
    pre_preds = tf.keras.layers.BatchNormalization()(pre_preds)
    pred = tf.keras.layers.Dense(1,activation='sigmoid')(pre_preds)
    model_full = tf.keras.models.Model(inputs= inputs,\
                                       outputs =pred)
    model_full.compile(loss=tf.keras.losses.binary_crossentropy,\
                       metrics=['accuracy'],
                       optimizer='adam')
    return model_full


In [21]:
def prepar_data_set(data_df):
    categoy_features = data_df.select_dtypes('category').columns
    numerique_features = data_df.select_dtypes('number').columns
    for col in categoy_features:
        encoder = LabelEncoder()
        data_df[col] = encoder.fit_transform(data_df[col])
    return data_df,categoy_features,numerique_features

In [22]:
train,cat_features,num_featture = prepar_data_set(vstr)